## Connect to ArcGIS Enterprise

The [GIS](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis) class is representative of a single ArcGIS Online organization or an ArcGIS Enterprise deployment. The `GIS` object provides helper objects to manage (search, create, retrieve) GIS resources such as content, users, and groups.

In [ ]:
# Import
from arcgis import GIS

If you are signing in frequently to a particular GIS and would like to [store the credentials locally](https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/) on your computer, then you could do so using the `profile` parameter of your `GIS`.

In [ ]:
# Connect to the ArcGIS Enterprise deployment
gis = GIS(profile="dev_summit_demo")

## Add data

The [ContentManager](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html?#contentmanager) class is a helper class for managing content in ArcGIS Online or ArcGIS Enterprise. An instance of this class, called [content](https://developers.arcgis.com/python/api-reference/arcgis.html?#arcgis.GIS.content), is available as a property of the GIS object. Users call methods on this `content` object to manipulate (create, get, search, etc) items.

The `.add()` method adds content to the GIS by creating an Item. The [Item](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html?#item) class represents an item in the GIS, where an item is simply considered a unit of content in the GIS.

In [ ]:
# Create a unique item name
import uuid
unique_sd_item_name = "seattle_example_tracks_demo_{0}".format(str(uuid.uuid4())[:6])
unique_sd_item_name

In [ ]:
# Add the service definition file
item = gis.content.add(item_properties={"title": unique_sd_item_name,
                                        "tags": "Seattle, Tracks"
                                       },
                       data=r"C:\Seattle_Example_Tracks\seattle_example_tracks.sd")

The [publish](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html?#arcgis.gis.Item.publish) method is used to publish a hosted service based on an existing source item (this item).

In [ ]:
# Publish the service definition file
published_item = item.publish()

The [shared_with](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html?#arcgis.gis.Item.shared_with) property reveals the privacy or sharing status of the current item.

In [ ]:
# View current sharing level
published_item.shared_with

The [share](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html?#arcgis.gis.Item.share) method shares an item with the specified list of groups.

In [ ]:
# Share the hosted feature layer with everyone
published_item.share(everyone=True)

In [ ]:
# View current sharing level
published_item.shared_with

## Search for data

The [search](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html?#arcgis.gis.ContentManager.search) method searches for portal items and returns a list of [Item](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html?#item) objects matching the specified query.

In [ ]:
# Find the Seattle example tracks data
seattle_example_tracks = gis.content.search(query="title:{0}".format(unique_sd_item_name),
                                            item_type="feature layer",
                                            max_items=1)[0]
seattle_example_tracks

In [ ]:
# List all of the Seattle example tracks layers
for layer in seattle_example_tracks.layers:
    print(layer.properties.name)

## Create Spatially Enabled DataFrames

The [Spatially Enabled DataFrame](https://developers.arcgis.com/python/guide/introduction-to-the-spatially-enabled-dataframe/) inserts a custom namespace called spatial into the popular [Pandas](https://pandas.pydata.org/) [DataFrame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) structure to give it spatial abilities. This allows you to use intutive, pandorable operations on both the attribute and spatial columns. Thus, the SEDF is based on data structures inherently suited to data analysis, with natural operations for the filtering and inspecting of subsets of values which are fundamental to statistical and geographic manipulations.

In [ ]:
# Imports
import pandas as pd

In [ ]:
# Create a Spatially Enabled DataFrame object
sdf = pd.DataFrame.spatial.from_layer(seattle_example_tracks.layers[4])
sdf.head()

## Plotting the Spatially Enabled DataFrame

The [map](https://developers.arcgis.com/python/api-reference/arcgis.html?#arcgis.GIS.map) method creates a map widget centered at the declared location.

In [ ]:
# Create a map of Seattle
map_one = gis.map(location="Seattle")
map_one

You can quickly visualize the points by calling the `plot` method off the DataFrame's spatial accessor and passing the map you created above.

In [ ]:
# Plot the Spatially Enabled DataFrame
sdf.spatial.plot(map_widget=map_one)